# Week 3 Submission By Eviatar Shemesh

## First Level - Getting And Cleaning The Data

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import display_html

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header = 0)[0]
df.columns = df.columns.str.replace(' ', '')
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
df.shape

(180, 3)

In [4]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
df.reset_index()
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
df.shape

(103, 3)

In [7]:
df.loc[df['Neighborhood'] == 'Not assigned']

,Postalcode,Borough,Neighborhood


In [8]:
df.shape

(103, 3)

## Second Level - Inserting Latitude And Longitude To Each Postal Code In Our DataFrame

In [9]:
df_postal = pd.read_csv('http://cocl.us/Geospatial_data')
df_postal.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df['Latitude'] = 'Not Assigned'
df['Longitude'] = 'Not Assigned'
for ind in df.index:
    PostalCode = df['Postalcode'][ind]
    row = df_postal.loc[df_postal['Postal Code'] == PostalCode ]
    df['Latitude'][ind] = row.iloc[0]['Latitude']
    df['Longitude'][ind] = row.iloc[0]['Longitude']
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7533,-79.3297
3,M4A,North York,Victoria Village,43.7259,-79.3156
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6543,-79.3606
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.7185,-79.4648
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6623,-79.3895


## Third Level - Cluster the neighborhoods in Toronto

In [11]:
import numpy as np
!pip install geopy
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
print('Libraries imported.')

Libraries imported.


In [12]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude_tor = location.latitude
longitude_tor = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_tor, longitude_tor))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
df_tor = df.copy()
df_tor.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.7533,-79.3297
3,M4A,North York,Victoria Village,43.7259,-79.3156
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6543,-79.3606
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.7185,-79.4648
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6623,-79.3895


In [14]:
df_tor['ContainsToronto'] = df['Borough'].str.contains(r'Toronto') #Search If Each Borough Contains String 'Toronto', And Drop The Rows That Don't Contain
indexNames = df_tor[df_tor['ContainsToronto'] == False ].index #The index of rows that don't contain String 'Toronto'
df_tor.drop(indexNames , inplace=True)
df_tor.drop(['ContainsToronto'], axis=1, inplace = True)
df_tor.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6543,-79.3606
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6623,-79.3895
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
22,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
30,M4E,East Toronto,The Beaches,43.6764,-79.293


In [15]:
map_toronto= folium.Map(location=[latitude_tor, longitude_tor], zoom_start=11)
for lat, lng, borough, PostalCode in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Postalcode']):
    label = '{}, {}'.format(PostalCode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
CLIENT_ID = 'IKVJZO3L2RKHD12H35LD3S40Z51JWXA4NN4XL3PH00GK2QRU' # your Foursquare ID
CLIENT_SECRET = 'FAMQK0S5IWNNSM00JNCZXYXLK4XUBIMSPY3JSWCFG0XE0XVP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 #How many vanues Per Postal Code
radius = 500 #Radius Of Search

In [17]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
#function that gets the nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names = df_tor['Postalcode'],
                                   latitudes = df_tor['Latitude'],
                                   longitudes = df_tor['Longitude']
                                  )


M5A
M7A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [20]:
toronto_venues.groupby('PostalCode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4E,5,5,5,5,5,5
M4K,42,42,42,42,42,42
M4L,21,21,21,21,21,21
M4M,42,42,42,42,42,42
M4N,4,4,4,4,4,4
M4P,8,8,8,8,8,8
M4R,17,17,17,17,17,17
M4S,36,36,36,36,36,36
M4T,2,2,2,2,2,2


In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Pc_venues_sorted = pd.DataFrame(columns=columns)
Pc_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    Pc_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [24]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
#add cluster labels
Pc_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_tor
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(Pc_venues_sorted.set_index('PostalCode'), on='Postalcode')
toronto_merged.head() 

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6543,-79.3606,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Mexican Restaurant,Restaurant,Farmers Market,Shoe Store
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6623,-79.3895,0,Coffee Shop,Yoga Studio,Creperie,Beer Bar,Boutique,Burger Joint,Burrito Place,Restaurant,Café,Park
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,0,Coffee Shop,Clothing Store,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Tea Room,Bookstore,Lingerie Store
22,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,0,Coffee Shop,Café,Restaurant,Gastropub,Italian Restaurant,Cocktail Bar,Beer Bar,Cosmetics Shop,Diner,Japanese Restaurant
30,M4E,East Toronto,The Beaches,43.6764,-79.293,4,Pub,Neighborhood,Coffee Shop,Health Food Store,Trail,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [25]:
map_clusters = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postalcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters